In [5]:

import re
import os
import sys
import boto3 as bt3
import s3fs
import pandas as pd
# import qgrid


In [6]:
from pyspark import *
from pyspark.sql import *
from IPython.display import display


In [8]:

pd.set_option('display.max_columns', 500)
pd.set_option('display.expand_frame_repr', False)
# pd.set_option('display.max_colwidth', -1)



In [9]:

# ====================================
# Data Loading

s3 = bt3.resource('s3')
my_bucket = s3.Bucket('tc-jp-bucket1')
s3_loc = 's3://tc-jp-bucket1'
data_loc_list = ['northwind-db/',
  ]



In [10]:
# Visual list of files
ls_loc = []
ls_file = []
for data_loc in data_loc_list:
  for obj in my_bucket.objects.filter(Prefix=data_loc):
    print(obj.key)
    file = s3_loc + obj.key
    ls_file.append(file)
  print(data_loc + '\n')
print("number of files:" + str(len(ls_file)))



northwind-db/
northwind-db/categories.csv
northwind-db/customers.csv
northwind-db/employee-territories.csv
northwind-db/employees.csv
northwind-db/order-details.csv
northwind-db/orders.csv
northwind-db/products.csv
northwind-db/regions.csv
northwind-db/shippers.csv
northwind-db/suppliers.csv
northwind-db/territories.csv
northwind-db/

number of files:12


In [13]:
spark = SparkSession.builder.appName('TC-APP1').master('local[*]') \
.config('spark.sql.hive.convertMetastoreParquet', 'false') \
.config('spark.sql.hive.caseSensitiveInferenceMode', 'NEVER_INFER') \
.enableHiveSupport() \
.getOrCreate()



In [ ]:
spark.sql("CREATE DATABASE IF NOT EXISTS dbpega COMMENT 'This is DB for pega data' ").show()
spark.sql("DESCRIBE DATABASE EXTENDED dbpega ").show()



In [14]:
v_source_file_loc_path = ["s3://a_s3_bucket/path/to/folder/"]

df_target1 = spark.read.format("parquet").load(v_source_file_loc_path)
df_target1.printSchema()


Py4JJavaError: An error occurred while calling o65.load.
: java.io.IOException: No FileSystem for scheme: s3
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:2660)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:2667)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:94)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:2703)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:2685)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:373)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:295)
	at org.apache.spark.sql.execution.datasources.DataSource$.org$apache$spark$sql$execution$datasources$DataSource$$checkAndGlobPathIfNecessary(DataSource.scala:709)
	at org.apache.spark.sql.execution.datasources.DataSource$$anonfun$15.apply(DataSource.scala:390)
	at org.apache.spark.sql.execution.datasources.DataSource$$anonfun$15.apply(DataSource.scala:390)
	at scala.collection.TraversableLike$$anonfun$fl

In [ ]:

# Temp tables are more resistant to a shared hive tables - avoids tampering
df_target1.registerTempTable("t_check_attrib")

df_target1.columns
display(df_target1.columns)